In [1]:
from gensim import corpora,models,similarities,utils
import jieba
import jieba.posseg as pseg
import re
import datetime
import random
import pandas as pd
import gensim

In [2]:
#廣告資料
ad_data_list = [['【末日之子】人氣RPG手遊','遊戲的故事發生在被大量隕石撞擊過後的末日世界，隕石上所遺留的外星病毒讓大量民眾感染成為喪屍，而人類最終兵器「滅絕之鎚」的攻擊不僅沒能徹底消滅它們，反而讓喪屍因此產生了變異，開始用各種詭異的特殊能力攻擊人類。',20],
               ['百萬下載 經典人氣手遊【楓之谷M】','《楓之谷M》回來啦！在手機上重新誕生的MMORPG經典《楓之谷M》，帶你找回最初的感動。',15],
               ['快下載享受變成龍打怪的快感','人與龍族並肩作戰的動作 RPG。「失落的龍絆」是以輕鬆滑動來操作的動作 RPG。使用與您締結契約的龍族之力而戰的「龍化」。華麗多樣的動作、技能與龍化時機都由您一手操作，擊敗種種強敵。',10],
               ['黑色沙漠 全新職業【魔女】','一款沙盒大型多人在線角色扮演遊戲，受到150多個國家喜愛的 MMORPG!在MOBILE也可以體驗黑色沙漠的魅力與感動',5],
               ['第五人格','網易首款非對稱性對抗競技手遊。荒誕哥德畫風，懸疑燒腦劇情，刺激的1V4非對稱的對抗玩法，都將給玩家帶來全新的遊戲體驗。玩家將成為偵探奧爾菲斯，在收到一封神秘的委託信後，進入惡名昭著的莊園調查一件失蹤案。在進行證據調查過程中，奧爾菲斯將採用演繹法，對案情進行回顧。在案情回顧時，玩家可以選擇扮演監管者或求生者，展開激烈的對抗。而在調查的過程，無限接近事實時，卻發現越來越不可思議的真相。',20]]

In [3]:
#讀取文章標題資料
news_data = pd.read_csv('all_news_title.csv', header=None)

In [8]:
news_data[0][1]

'台北東區美髮沙龍推薦→Moon hair studio 月穆髮型藝術~擺脫男人婆，變身甜美歐妮！'

In [5]:
ad_title_list = []
for i in range(len(ad_data_list)):
    ad_title_list.append(ad_data_list[i][0])

In [9]:
news_title_list = []
for i in range(len(news_data[0])):
    news_title_list.append(news_data[0][i])

In [10]:
#移除括號內的內容
remover1=re.compile(r"\（.*?\）")
remover2=re.compile(r"\[.*?\]")
remover3=re.compile(r"\〔.*?\〕")
remover4=re.compile(r"\(.*?\)")

for i in range(len(news_title_list)):
        news_title_list[i]=remover1.sub("",news_title_list[i])
        news_title_list[i]=remover2.sub("",news_title_list[i])
        news_title_list[i]=remover3.sub("",news_title_list[i])
        news_title_list[i]=remover4.sub("",news_title_list[i])
        
for i in range(len(ad_title_list)):
        ad_title_list[i]=remover1.sub("",ad_title_list[i])
        ad_title_list[i]=remover2.sub("",ad_title_list[i])
        ad_title_list[i]=remover3.sub("",ad_title_list[i])
        ad_title_list[i]=remover4.sub("",ad_title_list[i])

In [11]:
#設停用字
with open('/Users/jack/Desktop/LDA/stopword.txt', encoding='utf-8') as f:
    stop_word_list = f.readlines()
    
stop_word_list = [x.strip() for x in stop_word_list]

以下分別以LDA及LSI做文章與廣告標題相似度分析
每次分析將以廣告標題加上要分析的文章標題做dictionary後建模

In [15]:
#LDA
lda_sims_result = []
for index in range(len(news_title_list)):
    texts_list = []
    ad_title_list.append(news_title_list[index])
    for i in ad_title_list:
        tokens = jieba.cut(i,cut_all=False,HMM=True)
        stopped_tokens = [i for i in tokens if not i in stop_word_list]
        texts_list.append(stopped_tokens)
        
    ad_title_list = ad_title_list[:-1]
    

    # 重建tfidf model
    dictionary = corpora.Dictionary(texts_list)
    corpus = [dictionary.doc2bow(text) for text in texts_list]
    tfidfModel = models.TfidfModel(corpus)
    # 轉為向量表示
    tfidfVectors = tfidfModel[corpus]

    # load lda model
    #ldamodel = models.ldamodel.LdaModel.load('/Users/jack/Desktop/LDA/another/mobile01_LDA_model.lda')
    lda_model = gensim.models.ldamodel.LdaModel(tfidfVectors,
                                                num_topics = 5,
                                                id2word = dictionary,
                                                chunksize = 2000,
                                                passes = 20)

    corpus_lda = lda_model.get_document_topics(tfidfVectors ,minimum_probability=0)
    # 建立索引
    #indexLDA = similarities.MatrixSimilarity(ldamodel[tfidfVectors])
    indexLDA = similarities.MatrixSimilarity(corpus_lda)
    
    sims = indexLDA[lda_model[dictionary.doc2bow(texts for texts in news_title_list[index])]]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print(index ,sims)
    lda_sims_result.append(sims)

0 [(5, 0.99995065), (0, 0.9974194), (4, 0.18306792), (2, 0.13917711), (3, 0.13294223), (1, 0.12865499)]
1 [(5, 0.99781525), (3, 0.9945876), (0, 0.17748962), (1, 0.16730502), (4, 0.15209542), (2, 0.10833239)]
2 [(5, 0.99773735), (4, 0.14633869), (0, 0.10706768), (2, 0.10514877), (3, 0.099289924), (1, 0.094961315)]
3 [(5, 0.9983184), (1, 0.9959106), (4, 0.98247695), (0, 0.11740109), (2, 0.11510365), (3, 0.1112182)]
4 [(5, 0.9988368), (1, 0.9949742), (4, 0.14918555), (0, 0.11007167), (2, 0.10777101), (3, 0.10209396)]
5 [(5, 0.9991646), (0, 0.99474823), (4, 0.16299486), (2, 0.12055754), (3, 0.11453241), (1, 0.11042466)]
6 [(5, 0.99853975), (2, 0.99241155), (4, 0.9793826), (0, 0.10717982), (3, 0.10139836), (1, 0.09679859)]
7 [(2, 0.9872733), (5, 0.97820175), (4, 0.3251673), (0, 0.27446815), (3, 0.2642511), (1, 0.2586087)]
8 [(5, 0.99845153), (4, 0.15276611), (0, 0.11304817), (2, 0.11107336), (3, 0.10716022), (1, 0.102494806)]
9 [(5, 0.9985056), (4, 0.17112982), (0, 0.13012263), (2, 0.128091

80 [(5, 0.99834365), (1, 0.9953388), (0, 0.9930823), (4, 0.15267006), (2, 0.110989906), (3, 0.10675869)]
81 [(5, 0.9978891), (1, 0.9976184), (4, 0.17107767), (0, 0.13045338), (2, 0.12804651), (3, 0.1246153)]
82 [(5, 0.9984234), (3, 0.995966), (4, 0.16305883), (0, 0.122552305), (2, 0.12055132), (1, 0.11159053)]
83 [(5, 0.99943256), (3, 0.99597), (4, 0.16300732), (0, 0.12258261), (2, 0.12065582), (1, 0.11175533)]
84 [(5, 0.9999974), (1, 0.9997771), (4, 0.2021441), (0, 0.15954517), (2, 0.15688206), (3, 0.15256342)]
85 [(5, 0.9994653), (3, 0.995968), (4, 0.9841036), (0, 0.122600414), (2, 0.12053915), (1, 0.11209341)]
86 [(5, 0.99801624), (2, 0.9919348), (4, 0.14411764), (0, 0.104941376), (3, 0.099129446), (1, 0.0945465)]
87 [(5, 0.99866855), (4, 0.14919232), (0, 0.10977224), (2, 0.10777741), (3, 0.101921596), (1, 0.097464986)]
88 [(5, 0.9998199), (2, 0.99927), (4, 0.202178), (0, 0.15911463), (3, 0.15050398), (1, 0.14567858)]
89 [(5, 0.9998151), (1, 0.9976281), (4, 0.17114264), (0, 0.130617

158 [(5, 0.9990127), (0, 0.9947835), (4, 0.16307136), (2, 0.12080315), (3, 0.11645388), (1, 0.11220653)]
159 [(5, 0.99999994), (4, 0.64935374), (0, 0.5896715), (2, 0.5863449), (3, 0.57735884), (1, 0.5706468)]
160 [(0, 0.9440421), (2, 0.9433012), (5, 0.9331531), (1, 0.47359756), (4, 0.2082242), (3, 0.14879376)]
161 [(5, 0.99997777), (4, 0.20262323), (0, 0.16017069), (2, 0.15795384), (3, 0.15320158), (1, 0.1485871)]
162 [(5, 0.9995316), (1, 0.99685675), (4, 0.16301137), (0, 0.12291916), (2, 0.12057449), (3, 0.11474804)]
163 [(5, 0.999384), (4, 0.20216407), (0, 0.1589602), (2, 0.15690598), (3, 0.15046947), (1, 0.14559123)]
164 [(5, 0.9999998), (4, 0.20209844), (0, 0.15897092), (2, 0.15822984), (3, 0.1503992), (1, 0.14568977)]
165 [(5, 0.9982395), (4, 0.15279959), (0, 0.11298591), (2, 0.111121334), (3, 0.106903456), (1, 0.102153994)]
166 [(5, 0.9998361), (4, 0.18309046), (0, 0.14185496), (2, 0.13917522), (3, 0.13535145), (1, 0.1303246)]
167 [(4, 0.6493205), (0, 0.5897345), (2, 0.58632433),

237 [(5, 0.99767053), (4, 0.14639927), (0, 0.10708708), (2, 0.10514054), (3, 0.1016648), (1, 0.09679012)]
238 [(5, 0.9995662), (4, 0.98907053), (0, 0.14118008), (2, 0.13909149), (3, 0.13282414), (1, 0.12818575)]
239 [(5, 0.9978147), (3, 0.9926256), (4, 0.14209202), (0, 0.10315209), (2, 0.10121224), (1, 0.09311059)]
240 [(5, 0.9771472), (0, 0.34775662), (1, 0.3376785), (4, 0.19409521), (2, 0.1433962), (3, 0.13859539)]
241 [(5, 0.9998782), (3, 0.9995611), (4, 0.20214531), (0, 0.1590392), (2, 0.1572113), (1, 0.14773487)]
242 [(5, 0.9975127), (4, 0.15272081), (0, 0.112944424), (2, 0.11104706), (3, 0.108129635), (1, 0.102794446)]
243 [(5, 0.99999976), (4, 0.2022476), (0, 0.15915745), (2, 0.15696943), (3, 0.15311047), (1, 0.14796625)]
244 [(5, 0.9989706), (1, 0.9967003), (4, 0.16305955), (0, 0.12323081), (2, 0.12051792), (3, 0.11760866)]
245 [(5, 0.99993706), (2, 0.9977321), (4, 0.18340309), (0, 0.14149289), (3, 0.1362961), (1, 0.1303717)]
246 [(5, 0.9986304), (4, 0.15267333), (0, 0.11295926

315 [(4, 0.64932996), (0, 0.5895153), (2, 0.5863278), (3, 0.5773338), (5, 0.5758747), (1, 0.5706998)]
316 [(3, 0.9996667), (1, 0.9994105), (5, 0.9978957), (4, 0.2376871), (0, 0.19269207), (2, 0.189971)]
317 [(5, 0.99983925), (2, 0.99930364), (4, 0.20244017), (0, 0.15990008), (3, 0.15297534), (1, 0.14804919)]
318 [(5, 0.9899003), (3, 0.98547274), (1, 0.984092), (4, 0.32499278), (0, 0.27454686), (2, 0.27168542)]
319 [(5, 0.99983186), (4, 0.18298855), (0, 0.14122121), (2, 0.13910289), (3, 0.13285609), (1, 0.12819014)]
320 [(5, 0.99926174), (4, 0.20203605), (0, 0.15898289), (2, 0.15679997), (3, 0.1503742), (1, 0.14554882)]
321 [(2, 0.98727006), (5, 0.98652214), (4, 0.32517338), (0, 0.27447623), (3, 0.26425853), (1, 0.25861496)]
322 [(4, 0.996877), (5, 0.9930693), (1, 0.9841671), (0, 0.27442855), (2, 0.27159598), (3, 0.26410362)]
323 [(4, 0.6493311), (0, 0.58957845), (2, 0.58638066), (5, 0.58637774), (3, 0.5773241), (1, 0.5707059)]
324 [(5, 0.9834085), (4, 0.32495242), (0, 0.2742794), (2, 0

...

10971 [(5, 0.99906045), (4, 0.1571164), (0, 0.117236264), (2, 0.11521413), (3, 0.11189772), (1, 0.106689736)]
10972 [(5, 0.99999833), (1, 0.9987149), (2, 0.99771386), (4, 0.1830928), (0, 0.14184427), (3, 0.13487104)]
10973 [(5, 0.9994489), (2, 0.99635255), (4, 0.17164925), (0, 0.13025656), (3, 0.124093294), (1, 0.11922808)]
10974 [(5, 0.99997944), (4, 0.9892927), (0, 0.14156231), (2, 0.13948348), (3, 0.13683541), (1, 0.13194324)]
10975 [(5, 0.99988663), (3, 0.9995599), (4, 0.2021369), (0, 0.15903515), (2, 0.15687206), (1, 0.14748213)]
10976 [(5, 0.9999482), (0, 0.9975152), (4, 0.18516323), (2, 0.13945399), (3, 0.13657132), (1, 0.13222148)]
10977 [(4, 0.6493624), (0, 0.5897204), (2, 0.58639675), (3, 0.5774023), (1, 0.5705144), (5, 0.5586369)]
10978 [(4, 0.64928806), (0, 0.5896966), (5, 0.58632386), (2, 0.5863236), (3, 0.5773492), (1, 0.5705178)]
10979 [(5, 0.9999864), (4, 0.9931541), (0, 0.1593777), (2, 0.1580202), (3, 0.15348524), (1, 0.14819229)]
10980 [(0, 0.8599132), (1, 0.85397196)

11050 [(3, 0.9996507), (5, 0.9975311), (4, 0.23789704), (0, 0.19248867), (2, 0.19014479), (1, 0.18063387)]
11051 [(5, 0.9999791), (2, 0.99779445), (4, 0.1852417), (0, 0.14154509), (3, 0.13664746), (1, 0.13177061)]
11052 [(5, 0.77836245), (1, 0.7253958), (4, 0.24646342), (0, 0.19062345), (2, 0.18736051), (3, 0.18201748)]
11053 [(4, 0.64932597), (0, 0.5895604), (2, 0.5863712), (3, 0.57732123), (1, 0.5706032), (5, 0.56750786)]
11054 [(2, 0.9998371), (3, 0.9996481), (5, 0.9981867), (4, 0.23795076), (0, 0.1925024), (1, 0.18080972)]
11055 [(5, 0.9999403), (0, 0.9991712), (4, 0.2036467), (2, 0.15732539), (3, 0.15372668), (1, 0.14890489)]
11056 [(5, 0.9979762), (2, 0.99428815), (4, 0.15723315), (0, 0.11717311), (3, 0.11131049), (1, 0.10662689)]
11057 [(5, 0.99967104), (1, 0.9976188), (4, 0.17112935), (0, 0.13048762), (2, 0.12809223), (3, 0.12380989)]
11058 [(5, 0.9999509), (3, 0.99825054), (4, 0.9890908), (0, 0.1413473), (2, 0.13913284), (1, 0.1300137)]
11059 [(3, 0.98465556), (4, 0.3276231), 

11127 [(5, 0.999894), (4, 0.18331906), (0, 0.14143276), (2, 0.1393573), (3, 0.13612801), (1, 0.13123704)]
11128 [(2, 0.99985087), (5, 0.99845207), (4, 0.9980243), (0, 0.19214994), (3, 0.18318346), (1, 0.17803381)]
11129 [(5, 0.9990566), (4, 0.9825424), (0, 0.11720158), (2, 0.115198165), (3, 0.110975645), (1, 0.10625714)]
11130 [(5, 0.9996199), (0, 0.9991958), (4, 0.20292406), (2, 0.1587426), (3, 0.15437067), (1, 0.14930636)]
11131 [(3, 0.98573196), (1, 0.98423946), (4, 0.32648706), (0, 0.27638474), (2, 0.27205685), (5, 0.25420505)]
11132 [(3, 0.985408), (5, 0.97911483), (4, 0.32611787), (0, 0.27524474), (2, 0.27262697), (1, 0.26429325)]
11133 [(5, 0.99987745), (4, 0.99298894), (0, 0.15899102), (2, 0.15685044), (3, 0.15038145), (1, 0.14562161)]
11134 [(1, 0.9838632), (5, 0.9783113), (4, 0.32946137), (0, 0.27569404), (2, 0.27279958), (3, 0.27063355)]
11135 [(5, 0.999603), (1, 0.99762094), (2, 0.9963066), (4, 0.17113352), (0, 0.1306706), (3, 0.12402869)]
11136 [(5, 0.99997276), (3, 0.9982

11205 [(5, 0.99792325), (4, 0.23928127), (0, 0.19357952), (3, 0.1922877), (2, 0.19128741), (1, 0.1869643)]
11206 [(5, 0.91577387), (0, 0.5356786), (2, 0.5340502), (1, 0.5261176), (4, 0.23506916), (3, 0.17343758)]
11207 [(5, 0.9998817), (4, 0.20257741), (0, 0.15937933), (2, 0.15803576), (3, 0.15346843), (1, 0.14820388)]
11208 [(1, 0.983376), (4, 0.3306647), (0, 0.28059754), (2, 0.27356195), (3, 0.27172527), (5, 0.23427662)]
11209 [(5, 0.99688786), (4, 0.23769726), (0, 0.19226539), (2, 0.18998379), (3, 0.18314801), (1, 0.17812109)]
11210 [(5, 0.9999548), (3, 0.99956125), (4, 0.20216607), (0, 0.15906477), (2, 0.15689968), (1, 0.1474975)]
11211 [(5, 0.9983647), (4, 0.2393476), (0, 0.1927838), (2, 0.19045281), (3, 0.18679266), (1, 0.18161803)]
11212 [(1, 0.9833744), (4, 0.33049595), (2, 0.27709353), (0, 0.27667207), (3, 0.27154335), (5, 0.24017417)]
11213 [(5, 0.98049587), (4, 0.32501203), (0, 0.27431935), (2, 0.2717122), (3, 0.26410657), (1, 0.2584543)]
11214 [(5, 0.99941045), (2, 0.996349

11283 [(5, 0.99963146), (4, 0.17108919), (0, 0.13009804), (2, 0.12806253), (3, 0.12195106), (1, 0.11739582)]
11284 [(2, 0.99976456), (5, 0.99797577), (4, 0.24129535), (0, 0.19348589), (3, 0.1889159), (1, 0.18304808)]
11285 [(4, 0.9961645), (1, 0.98297083), (0, 0.28129357), (2, 0.27802846), (3, 0.27246368), (5, 0.25385424)]
11286 [(4, 0.64937675), (0, 0.5895706), (2, 0.58633316), (3, 0.57729715), (1, 0.5708331), (5, 0.5606812)]
11287 [(2, 0.999748), (5, 0.99722457), (4, 0.23944733), (0, 0.19606158), (3, 0.18965809), (1, 0.18331805)]
11288 [(5, 0.99993086), (4, 0.20237657), (0, 0.1596924), (2, 0.15706713), (3, 0.15316188), (1, 0.14786386)]
11289 [(4, 0.6493723), (0, 0.5897039), (2, 0.58637977), (3, 0.57841), (1, 0.5706774), (5, 0.56175077)]
11290 [(0, 0.99982095), (5, 0.99718076), (4, 0.23962441), (2, 0.19154893), (3, 0.1897664), (1, 0.18481836)]
11291 [(5, 0.9999979), (4, 0.20257029), (0, 0.15936765), (2, 0.15721887), (3, 0.15443896), (1, 0.1494031)]
11292 [(5, 0.99951845), (2, 0.995245

In [62]:
news_title_list[3047]

'中年大叔請教理財'

In [43]:
#以相關性第二高做排列（因第一高通常是自己對自己)
lda_sims_result_resort = sorted(enumerate(lda_sims_result), key=lambda x: x[1][1], reverse=True)

In [44]:
lda_sims_result_resort

[(3047,
  [(0, 0.9999081),
   (5, 0.99985313),
   (1, 0.9993241),
   (4, 0.23762918),
   (2, 0.18991844),
   (3, 0.18327853)]),
 (655,
  [(2, 0.9998531),
   (5, 0.9998531),
   (3, 0.99960154),
   (4, 0.23763086),
   (0, 0.19209734),
   (1, 0.17811221)]),
 (2732,
  [(0, 0.99991596),
   (5, 0.9998531),
   (4, 0.9980158),
   (2, 0.189901),
   (3, 0.18311474),
   (1, 0.1783732)]),
 (10074,
  [(1, 0.9998165),
   (5, 0.99980634),
   (4, 0.2021295),
   (0, 0.15948212),
   (2, 0.15686995),
   (3, 0.15058693)]),
 (3541,
  [(1, 0.99981385),
   (5, 0.9998063),
   (4, 0.20212226),
   (0, 0.15935165),
   (2, 0.15686874),
   (3, 0.15058622)]),
 (6542,
  [(1, 0.9998164),
   (5, 0.9998063),
   (4, 0.20212637),
   (0, 0.15949777),
   (2, 0.15687235),
   (3, 0.15063408)]),
 (9976,
  [(1, 0.999815),
   (5, 0.9998063),
   (4, 0.2021301),
   (0, 0.1593646),
   (2, 0.15687633),
   (3, 0.15066823)]),
 (1126,
  [(1, 0.9998148),
   (5, 0.999806),
   (2, 0.99926704),
   (4, 0.20216793),
   (0, 0.15941869),
   (

In [17]:
#LSI
lsi_sims_result = []
for index in range(len(news_title_list)):
    texts_list = []
    ad_title_list.append(news_title_list[index])
    for i in ad_title_list:
        tokens = jieba.cut(i,cut_all=False,HMM=True)
        stopped_tokens = [i for i in tokens if not i in stop_word_list]
        texts_list.append(stopped_tokens)
        
    ad_title_list = ad_title_list[:-1]
    

    # 重建tfidf model
    dictionary = corpora.Dictionary(texts_list)
    corpus = [dictionary.doc2bow(text) for text in texts_list]
    tfidfModel = models.TfidfModel(corpus)
    # 轉為向量表示
    tfidfVectors = tfidfModel[corpus]
    
    #建立lsi model
    lsi_model = models.LsiModel(tfidfVectors, id2word=dictionary, num_topics=5)
    

    #corpus_lsi = lda_model.get_document_topics(tfidfVectors ,minimum_probability=0)
    # 建立索引
    #indexLDA = similarities.MatrixSimilarity(ldamodel[tfidfVectors])
    indexLSI = similarities.MatrixSimilarity(lsi_model[corpus])
    
    sims = indexLSI[lsi_model[dictionary.doc2bow(texts for texts in news_title_list[index])]]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print(index ,sims)
    lsi_sims_result.append(sims)

0 [(5, 1.0), (4, 2.9802322e-08), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]
1 [(5, 0.98438), (1, 0.39965764), (3, 0.24789336), (0, 0.08976479), (2, 0.0), (4, 0.0)]
2 [(5, 1.0), (2, 2.9802322e-08), (4, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0)]
3 [(5, 0.99298966), (1, 0.4099294), (3, 0.3110243), (2, 0.0), (4, 0.0), (0, -0.02454532)]
4 [(5, 0.99999994), (4, 1.9557774e-08), (2, 3.7252903e-09), (0, 0.0), (1, 0.0), (3, 0.0)]
5 [(5, 1.0000001), (2, 1.4901161e-08), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)]
6 [(5, 0.99929595), (1, 0.099136464), (3, 0.08028668), (0, 0.0017137975), (2, 0.0), (4, 0.0)]
7 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
8 [(5, 0.9989369), (1, 0.11506373), (3, 0.09436083), (0, 0.0034287348), (2, 0.0), (4, 0.0)]
9 [(5, 0.9896428), (1, 0.4679426), (3, 0.36000127), (2, 0.0), (4, 0.0), (0, -0.025257617)]
10 [(5, 0.999949), (1, 0.07015313), (3, 0.04913813), (2, 0.0), (4, 0.0), (0, -0.008254379)]
11 [(5, 0.98377824), (1, 0.40013617), (3, 0.33576787), (0, 0.0184

112 [(5, 0.9485453), (1, 0.40764824), (0, 0.25878322), (3, 0.20114857), (2, 0.0), (4, 0.0)]
113 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
114 [(5, 0.99987495), (1, 0.64762425), (3, 0.46782476), (2, 0.0), (4, 0.0), (0, -0.17019843)]
115 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -1.1044904e-08)]
116 [(5, 0.9722373), (3, 0.25896144), (1, 0.2360909), (0, 0.08774354), (2, 0.0), (4, 0.0)]
117 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0), (2, -2.9802322e-08)]
118 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (4, -9.313226e-10), (2, -5.5879354e-09)]
119 [(5, 0.98885894), (3, 0.18223408), (1, 0.17788582), (0, 0.050199106), (2, 0.0), (4, 0.0)]
120 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
121 [(5, 0.7081516), (3, 0.6919208), (1, 0.5655171), (0, 0.29916748), (2, 0.0), (4, 0.0)]
122 [(5, 0.9133905), (3, 0.47938627), (1, 0.45306188), (0, 0.1464569), (2, 0.0), (4, 0.0)]
123 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
124 [(5, 1.0), 

210 [(5, 0.9997219), (1, 0.4191481), (3, 0.21633726), (2, 0.0), (4, 0.0), (0, -0.1174209)]
211 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (2, -1.4901161e-08), (4, -2.9802322e-08)]
212 [(5, 1.0), (4, 5.9604645e-08), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]
213 [(5, 0.9675541), (3, 0.28906316), (1, 0.26963535), (0, 0.09194727), (2, 0.0), (4, 0.0)]
214 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -7.450581e-09)]
215 [(5, 0.99999994), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -2.2351742e-08)]
216 [(5, 0.9579412), (1, 0.495634), (3, 0.32864013), (0, 0.16727066), (2, 0.0), (4, 0.0)]
217 [(5, 0.82515687), (3, 0.61090285), (1, 0.54003006), (0, 0.22377966), (2, 0.0), (4, 0.0)]
218 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (4, -6.0535967e-09), (2, -7.450581e-09)]
219 [(5, 0.9906518), (1, 0.56121886), (3, 0.4218408), (2, 0.0), (4, 0.0), (0, -0.06065376)]
220 [(5, 0.9276253), (1, 0.5022814), (3, 0.31833154), (0, 0.2630867), (2, 0.0), (4, 0.0)]
221 [(5, 0.9951674), (1, 0.60042447), (3, 0.43

306 [(5, 1.0), (2, 2.9802322e-08), (4, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0)]
307 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0), (2, -1.4901161e-08)]
308 [(5, 1.0), (2, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0), (4, -2.9802322e-08)]
309 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
310 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
311 [(5, 1.0), (4, 1.4901161e-08), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]
312 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -2.9802322e-08)]
313 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0), (2, -2.9802322e-08)]
314 [(5, 1.0), (4, 1.4901161e-08), (0, 0.0), (1, 0.0), (3, 0.0), (2, -2.9802322e-08)]
315 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
316 [(5, 0.92524815), (3, 0.40477443), (1, 0.35816982), (0, 0.14787218), (2, 0.0), (4, 0.0)]
317 [(5, 0.9908947), (1, 0.2469956), (3, 0.219118), (0, 0.027579874), (2, 0.0), (4, 0.0)]
318 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -2.

426 [(5, 1.0000001), (4, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0), (2, -2.9802322e-08)]
427 [(5, 1.0), (4, 3.7252903e-09), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]
428 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
429 [(5, 0.99881893), (1, 0.15364906), (3, 0.11977962), (2, 0.0), (4, 0.0), (0, -0.003193669)]
430 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
431 [(5, 1.0), (2, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)]
432 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
433 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
434 [(5, 0.99999994), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -2.9802322e-08)]
435 [(5, 1.0), (2, 4.4703484e-08), (0, 0.0), (1, 0.0), (3, 0.0), (4, -2.9802322e-08)]
436 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
437 [(5, 0.996395), (1, 0.16677341), (3, 0.14516431), (0, 0.0152967125), (2, 0.0), (4, 0.0)]
438 [(5, 1.0), (2, 2.9802322e-08), (4, 1.4901161e-08), (0, 0.0), (1, 0.0), (

...

10896 [(5, 0.900465), (3, 0.44277698), (1, 0.37728417), (0, 0.1760206), (2, 0.0), (4, 0.0)]
10897 [(5, 0.97399294), (1, 0.38578573), (3, 0.350093), (0, 0.051836733), (2, 0.0), (4, 0.0)]
10898 [(5, 0.9709179), (3, 0.28045368), (1, 0.2658323), (0, 0.08504572), (2, 0.0), (4, 0.0)]
10899 [(5, 0.94275385), (3, 0.3745895), (1, 0.3443584), (0, 0.12413762), (2, 0.0), (4, 0.0)]
10900 [(5, 0.9810387), (1, 0.25490075), (3, 0.2521312), (0, 0.060760953), (2, 0.0), (4, 0.0)]
10901 [(3, 0.7344875), (5, 0.6291572), (1, 0.5816562), (0, 0.33578134), (2, 0.0), (4, 0.0)]
10902 [(5, 0.9688988), (1, 0.38653886), (3, 0.36070105), (0, 0.064513706), (2, 0.0), (4, 0.0)]
10903 [(5, 0.9950955), (1, 0.16802861), (3, 0.1525909), (0, 0.02325438), (2, 0.0), (4, 0.0)]
10904 [(5, 0.9810387), (1, 0.25490075), (3, 0.2521312), (0, 0.060760953), (2, 0.0), (4, 0.0)]
10905 [(5, 0.9941797), (1, 0.1854079), (3, 0.16769618), (0, 0.024801817), (2, 0.0), (4, 0.0)]
10906 [(5, 0.9784453), (1, 0.34700853), (3, 0.31612387), (0, 0.048

10993 [(3, 0.76746476), (1, 0.595094), (5, 0.5556567), (0, 0.36315754), (2, 0.0), (4, 0.0)]
10994 [(5, 0.9941797), (1, 0.1854079), (3, 0.16769618), (0, 0.024801817), (2, 0.0), (4, 0.0)]
10995 [(5, 0.94275385), (3, 0.3745895), (1, 0.3443584), (0, 0.12413762), (2, 0.0), (4, 0.0)]
10996 [(5, 0.99010706), (1, 0.19054385), (3, 0.1862602), (0, 0.04271593), (2, 0.0), (4, 0.0)]
10997 [(5, 0.99575245), (1, 0.16732176), (3, 0.14892125), (0, 0.019408658), (2, 0.0), (4, 0.0)]
10998 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568), (2, 0.0), (4, 0.0)]
10999 [(5, 0.974292), (3, 0.27145463), (1, 0.26207072), (0, 0.0776185), (2, 0.0), (4, 0.0)]
11000 [(5, 1.0), (0, 0.0), (1, 0.0), (3, 0.0), (2, -1.44355e-08), (4, -2.9802322e-08)]
11001 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568), (2, 0.0), (4, 0.0)]
11002 [(5, 0.9901546), (1, 0.48224878), (3, 0.36741605), (2, 0.0), (4, 0.0), (0, -0.032315344)]
11003 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568

11083 [(5, 0.9675541), (3, 0.28906316), (1, 0.26963535), (0, 0.09194727), (2, 0.0), (4, 0.0)]
11084 [(5, 0.99122304), (1, 0.21107508), (3, 0.19540906), (0, 0.033776116), (2, 0.0), (4, 0.0)]
11085 [(5, 0.98784244), (1, 0.21480323), (3, 0.20871872), (0, 0.04658541), (2, 0.0), (4, 0.0)]
11086 [(5, 0.92524815), (3, 0.40477443), (1, 0.35816982), (0, 0.14787218), (2, 0.0), (4, 0.0)]
11087 [(5, 0.9339056), (3, 0.39025334), (1, 0.3513459), (0, 0.13663514), (2, 0.0), (4, 0.0)]
11088 [(5, 0.951777), (3, 0.3576099), (1, 0.33730662), (0, 0.11006166), (2, 0.0), (4, 0.0)]
11089 [(5, 0.96094215), (3, 0.33910033), (1, 0.330384), (0, 0.09394108), (2, 0.0), (4, 0.0)]
11090 [(5, 0.974292), (3, 0.27145463), (1, 0.26207072), (0, 0.0776185), (2, 0.0), (4, 0.0)]
11091 [(5, 0.908528), (3, 0.43092942), (1, 0.37115443), (0, 0.16741593), (2, 0.0), (4, 0.0)]
11092 [(5, 1.0), (2, 2.9802322e-08), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)]
11093 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568), (2, 0.0

11187 [(5, 0.98784244), (1, 0.21480323), (3, 0.20871872), (0, 0.04658541), (2, 0.0), (4, 0.0)]
11188 [(5, 0.8775282), (3, 0.56470495), (1, 0.53015935), (0, 0.17549747), (2, 0.0), (4, 0.0)]
11189 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568), (2, 0.0), (4, 0.0)]
11190 [(5, 0.95756924), (3, 0.31286442), (1, 0.28099802), (0, 0.11018072), (2, 0.0), (4, 0.0)]
11191 [(5, 0.9923731), (1, 0.17188546), (3, 0.16646162), (0, 0.03661217), (2, 0.0), (4, 0.0)]
11192 [(5, 0.9931827), (1, 0.18647704), (3, 0.17253333), (0, 0.029737882), (2, 0.0), (4, 0.0)]
11193 [(5, 0.98784244), (1, 0.21480323), (3, 0.20871872), (0, 0.04658541), (2, 0.0), (4, 0.0)]
11194 [(5, 0.99876773), (1, 0.13391924), (3, 0.107895195), (0, 0.0015820563), (2, 0.0), (4, 0.0)]
11195 [(3, 0.79286027), (1, 0.6059046), (5, 0.48848695), (0, 0.38375682), (2, 0.0), (4, 0.0)]
11196 [(5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, -9.313226e-10)]
11197 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568

11284 [(5, 0.9339056), (3, 0.39025334), (1, 0.3513459), (0, 0.13663514), (2, 0.0), (4, 0.0)]
11285 [(3, 0.7475633), (5, 0.6015412), (1, 0.5868967), (0, 0.34672838), (2, 0.0), (4, 0.0)]
11286 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
11287 [(5, 0.91678846), (3, 0.41829428), (1, 0.36478037), (0, 0.1580762), (2, 0.0), (4, 0.0)]
11288 [(5, 0.93105537), (3, 0.45559743), (1, 0.44709793), (0, 0.122579366), (2, 0.0), (4, 0.0)]
11289 [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]
11290 [(5, 0.91678846), (3, 0.41829428), (1, 0.36478037), (0, 0.1580762), (2, 0.0), (4, 0.0)]
11291 [(5, 0.9843849), (1, 0.2516869), (3, 0.24171555), (0, 0.051020034), (2, 0.0), (4, 0.0)]
11292 [(5, 0.9950955), (1, 0.16802861), (3, 0.1525909), (0, 0.02325438), (2, 0.0), (4, 0.0)]
11293 [(5, 0.9809219), (3, 0.2327393), (1, 0.22390433), (0, 0.06733337), (2, 0.0), (4, 0.0)]
11294 [(3, 0.7752155), (1, 0.5983538), (5, 0.53631246), (0, 0.36948568), (2, 0.0), (4, 0.0)]
11295 [(5, 0.9339056), (

In [63]:
news_title_list[9635]

'《第五人格》狂歡萬聖節！以莊園為主場大家一起變裝開趴'

In [38]:
lsi_sims_result[0][1][1]

2.9802322e-08

In [39]:
#以相關性第二高做排列（因第一高通常是自己對自己)
lsi_sims_result_resort = sorted(enumerate(lsi_sims_result), key=lambda x: x[1][1], reverse=True)

In [40]:
lsi_sims_result_resort

[(9635, [(4, 1.0), (5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]),
 (10466, [(4, 1.0), (5, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]),
 (9575,
  [(1, 0.79502726),
   (5, 0.7667585),
   (0, 0.49819735),
   (3, 0.45651895),
   (2, 0.0),
   (4, 0.0)]),
 (810,
  [(1, 0.7986023),
   (5, 0.7540629),
   (0, 0.51082534),
   (3, 0.46183804),
   (2, 0.0),
   (4, 0.0)]),
 (9909,
  [(1, 0.8051819),
   (5, 0.7303207),
   (0, 0.5328852),
   (3, 0.47131792),
   (2, 0.0),
   (4, 0.0)]),
 (844,
  [(1, 0.7559507),
   (5, 0.729939),
   (0, 0.6165653),
   (3, 0.28178722),
   (2, 0.0),
   (4, 0.0)]),
 (3452,
  [(1, 0.7374928),
   (5, 0.7264439),
   (0, 0.5728661),
   (3, 0.38104048),
   (2, 0.0),
   (4, 0.0)]),
 (7453,
  [(1, 0.7374928),
   (5, 0.7264439),
   (0, 0.5728661),
   (3, 0.38104048),
   (2, 0.0),
   (4, 0.0)]),
 (8176,
  [(1, 0.7374928),
   (5, 0.7264439),
   (0, 0.5728661),
   (3, 0.38104048),
   (2, 0.0),
   (4, 0.0)]),
 (10980,
  [(1, 0.7374928),
   (5, 0.7264439),
   (0, 0.5728661),
